Import everything

In [1]:
import sys
sys.path.append("../utils")
import utils_selection as utils_sel
import utils_data as utils_dat
import utils_bokeh as utils_bok
import glob
from pathlib import Path, WindowsPath

import pandas as pd
import numpy as np

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.server.server import Server
from tornado.ioloop import IOLoop
import asyncio
#output_notebook()

Set user parameters, data etc....

In [2]:
inputDir   = '/Users/helsens/Software/github/EPFL-TOP/singlecells_notebooks/data'
inputFiles = ['data_wscepfl0080.json', 'data_bleb001.json', 'data_bleb002.json']
outputFile = 'myFile_curatedPeaks.json'

selected_positions = {
#    'wscepfl0080':{'wscepfl0080_well1':['xy01','xy06'],
#                   'well2':['41','44']},
    'bleb001':{'well1':['xy'],
               'well2':['fna-bleb001_xy071.nd2','074'],
               'well3':['145','159'],
               'well4':['xy']},
    
        'bleb002':{'well1':['xy'],
               'well2':['xy'],
               'well3':['xy']}
}

Merge inputs and apply filters

In [3]:
analysis_data = utils_sel.merge_inputs(inputFiles, inputDir)

analysis_data_selected = utils_sel.select_data(analysis_data, selected_positions)

analysis_data_selected = utils_sel.select_main_flags(analysis_data_selected, ('keep_sample', True))
analysis_data_selected = utils_sel.select_main_flags(analysis_data_selected, ('sample_quality', 'High'))
analysis_data_selected = utils_sel.select_main_flags(analysis_data_selected, ('peaks_tod_div_validated', True))

analysis_data_selected = utils_sel.select_cell_flags(analysis_data_selected, ('dividing', False))
#analysis_data_selected = utils_sel.select_cell_flags(analysis_data_selected, ('dividing', True, 'alo'))
analysis_data_selected = utils_sel.select_cell_flags(analysis_data_selected, ('elongated', False))
analysis_data_selected = utils_sel.select_cell_flags(analysis_data_selected, ('oscillating', True), 'alo') #Two possible modes, "all" (default, all time frame must comply),  "alo" (at least one time frame must comply)

#Time in minutes "gt"=greater than, "lt"=less than, "eq" equal
analysis_data_selected = utils_sel.select_cell_main_features(analysis_data_selected, ('time_of_death', 200), 'gt')
analysis_data_selected = utils_sel.select_cell_main_features(analysis_data_selected, ('start_oscillation_time', 200), 'lt')
#analysis_data_selected = utils_sel.select_cell_main_features(analysis_data_selected, ('n_oscillations', 1), 'gt')



Convert to a dictionnary of intensities

In [4]:
intensities = utils_dat.get_intensities_time(analysis_data_selected)
peaks_tod   = utils_dat.get_peaks_tod(analysis_data_selected)

last_peaks_time = utils_dat.get_last_peaks_time(peaks_tod)
start_osc_time  = utils_dat.get_peaks_stuff_time('start_oscillation_time' ,peaks_tod)
end_osc_time  = utils_dat.get_peaks_stuff_time('end_oscillation_time',peaks_tod)
time_of_death   = utils_dat.get_peaks_stuff_time('time_of_death', peaks_tod)

utils_bok.intensities=intensities
utils_bok.peaks=peaks_tod
utils_bok.last_peaks_time = last_peaks_time
utils_bok.start_osc_time = start_osc_time
utils_bok.end_osc_time = end_osc_time
utils_bok.time_of_death = time_of_death

In [5]:
# Create the application
app = Application(FunctionHandler(utils_bok.modify_doc))

current_socket = 'localhost:8888'
port = 5005

#########################
# Integrate with the current Jupyter server -- To be checked
server = Server({'/': app}, port=port, io_loop=IOLoop.current(), allow_websocket_origin=[current_socket, "localhost:"+str(port)])

async def show_app():
    server.io_loop.add_callback(server.show, "/")
    await server.io_loop.start()

# Integrate with the Jupyter notebook event loop
loop = asyncio.get_event_loop()
if loop.is_running():
    loop.create_task(show_app())
else:
    loop.run_until_complete(show_app())


In [6]:
for exp in analysis_data_selected:
    print(exp)
    for well in analysis_data_selected[exp]:
        print('   ',well)
        for pos in analysis_data_selected[exp][well]:
            
            print('        ',pos)

wscepfl0080
    wscepfl0080_well2
         wscepfl0080_xy41.nd2
         wscepfl0080_xy44.nd2
bleb001
    bleb001_well4
         fna-bleb001_xy181.nd2
         fna-bleb001_xy183.nd2
         fna-bleb001_xy185.nd2
         fna-bleb001_xy187.nd2
         fna-bleb001_xy190.nd2
         fna-bleb001_xy195.nd2
         fna-bleb001_xy198.nd2
         fna-bleb001_xy199.nd2
         fna-bleb001_xy200.nd2
         fna-bleb001_xy211.nd2
         fna-bleb001_xy213.nd2
         fna-bleb001_xy220.nd2
    bleb001_well2
         fna-bleb001_xy074.nd2
    bleb001_well1
         fna-bleb001_xy002.nd2
         fna-bleb001_xy006.nd2
         fna-bleb001_xy007.nd2
         fna-bleb001_xy008.nd2
         fna-bleb001_xy031.nd2
         fna-bleb001_xy035.nd2
         fna-bleb001_xy037.nd2
         fna-bleb001_xy052.nd2
         fna-bleb001_xy053.nd2
         fna-bleb001_xy057.nd2
         fna-bleb001_xy062.nd2
